## 8. Traits
### 8.1 Traits

* traits are specific to OOP in Rust programming
* __Rust does not have inheritance__ like other programming langauges
* if two objects should share the same or a very similar method, this is called a trait
* `trait` allows to create an abstract parent class with certain features that can be shared by the child classes
  * these features can be implemented as "empty" just by their signature
  * or with a default function
* then these `trait`s can passed along to a `struct` (class) via the `impl`
  * does "inheritance" is explicitly modelled rather than implictily passed
  * to produce more `struct`s on the fly, a `trait` can also have a static method called `create() -> Self` (a factory function)
* instances of classes can then be instantiated
  * from `struct` 
  * from the factory method
  * from the trait with typing


Example
* code the following example with Animal as `trait`
* implement its methods onto the `struct`s Human and Cat

```consol
Animal
├── create() -> Self
├── name 
└── talk()

Human
├── create() -> self
├── name
└── talk()

Cat
├── create() -> self
├── name
└── talk()
```

In [2]:
// abstract-level Animal
trait Animal
{
    fn create(name: &'static str) -> Self;

    fn name(&self) -> &'static str;

    fn talk(&self)
    {println!("{} cannot talk", self.name());}
}

// class-level Human
struct Human
{
    name: &'static str
}

impl Animal for Human
{
    fn create(name: &'static str) -> Human
    {Human{name: name}}
    
    fn name(&self) -> &'static str
    {return self.name}

    fn talk(&self)
    {println!("{} says hello!", self.name());}
}

// class-level Cat
struct Cat
{
    name: &'static str
}

impl Animal for Cat
{
    fn create(name: &'static str) -> Cat
    {Cat{name: name}}

    fn name(&self) -> &'static str
    {return self.name}

    fn talk(&self)
    {println!("{} says meow!", self.name());}
}

fn traits_demo()
{
    // instantiate from struct
    let h = Human{name: "Tom"};
    h.talk();
    let c1 = Cat{name: "Gato"};
    c1.talk();
    // instantiate from factory
    let c2 = Cat::create("Katze");
    c2.talk();
    // instantiate from trait
    let h2: Human = Animal::create("John");
    h2.talk();
}

traits_demo()

Tom says hello!
Katze says meow!
John says hello!
Gato says meow!


()

* we can also use `trait` + `impl` to monkey patch additional functions onto existing generic types
* let's add a summation to the generic vector 

In [3]:
trait Summable<T>
{
    fn sum(&self) -> T;
}

impl Summable<i32> for Vec<i32>
{
    fn sum(&self) -> i32
    {
        let mut result:i32 = 0;
        for x in self{result += *x;}
        return result;
    }
}

let a = vec![1, 2, 3, 4];
println!("sum of a = {}", a.sum())

sum of a = 10


()

### 8.2 Trait Parameter

* now we can use `trait`s as parameters in functions
* there are 3 different ways to specify a `trait` parameter:
1. with an `impl` & `trait` in the signature:
  * `fn func(shape: impl Shape + Debug)`
2. with generic type & `trait` in the signature:
  * `fn print_info<T: Shape + Debug>(shape: T)`
3. with `where` clause in the function signature:
  * `fn print_info<T>(shape: T) where T: Shape + Debug`


Example
* create 2 simple geometric shapes: a Circle and a Square (as `struct`s) 
* create the `trait` Shape with an Area method
* create `impl`ementations of the Area `trait` for each Shape
  * $A_{square} = side ^2$
  * $A_{circle} = radius ^2 * \pi$

In [4]:
use std::fmt::Debug;

// our 2 simple shapes
#[derive(Debug)]
struct Circle {radius: f64,}
#[derive(Debug)]
struct Square {side: f64,}

// trait for Shape with Area
trait Shape
{
    fn area(&self) -> f64;
}

// implement area for square
impl Shape for Square {
    fn area(&self) -> f64 {
        self.side.powi(2)
    }
}

// implement area for circle
impl Shape for Circle {
    fn area(&self) -> f64 {
        self.radius.powi(2) * std::f64::consts::PI
    }
}

In [5]:
//fn print_info(shape: impl Shape + Debug)
//fn print_info<T: Shape + Debug>(shape: T)
fn print_info<T>(shape: T)
  where T: Shape + Debug
{
  println!("{:?}", shape);
  println!("The area is {}", shape.area());
}

fn trait_param_demo() {
  let c = Circle { radius: 2.0 };
  print_info(c);
  let sq = Square { side: 4.0 };
  print_info(sq);
}

trait_param_demo()

Circle { radius: 2.0 }
Square { side: 4.0 }
The area is 16
The area is 12.566370614359172


()

### 8.3 Into

* `Into` is powerful build in `trait` that helps transfer data between data types, objects & collections
* `Into` allows for automatic type conversion 
* it can be build in 2 different ways
1. with generic type & `trait` in the function signature:
* `fn new<S: Into<String>>(name: S) -> Person`
2. with the `where` clause in the function signature
* `fn new<S>(name: S) -> Person where S: Into<String>` 

In [6]:
struct Person {name: String}

impl Person
{
    // fn new(name: &str) -> Person
    // {
    //     Person{name: name.to_string()}
    // }
    fn new<S: Into<String>>(name: S) -> Person
    {
        Person{name: name.into()}
    }
}

fn into_demo(){
    // directly instantiating
    let john = Person::new("John");
    // indirectly instantiating
    let name = "Jane".to_string();
    // no forced conversion with as_ref() thanks to Into
    let jane = Person::new(name/*.as_ref()*/); 
}

into_demo()

()

### 8.4 Drop

* `Drop` is another simple but powerful `trait`
* it is a destructor (similar to other language)
* the `drop()` method gets automatically invoked at the end of the scope (where instance was created)

In [7]:
struct Creature{name: String}

impl Creature{
    fn new(name: &str) -> Creature{
        println!("{} enters the game.", name);
        Creature {name: name.into()}
    }
}

impl Drop for Creature{
    fn drop(&mut self){
        println!("{} is dead.", self.name);
    }
}

In [8]:
fn drop_demo()
{
    let goblin = Creature::new("Goblin");
    println!("Game procceeds...");
}

drop_demo()

Goblin enters the game.
Game procceeds...
Goblin is dead.


()

### 8.5 Operator Overloading

* operator overloading is done using `trait`s in Rust
* operators are taken from standard library 'std::ops'

Example
* create a `struct` for Complex numbers (consisting of a real part & an imaginary part)
* with any further implementation, you cannot add these Complex numbers together
  * compiler will `note: the trait 'Add' must be implemented`
  * addition of complex number is done by the component elements
* building a operators between two objects of the same type follows template of
  * `self plus other` or `self minus other`, etc.
  * `rhs` (right-hand side) or `other` are used in Rust

In [9]:
#[allow(unused)]
use std::ops::{Add};
use std::fmt::Debug;

// type for complex numbers
#[derive(Debug)] // #[derive(PartialEq, Eq, PartialOrd, Ord)] 
struct Complex<T>
{
    real: T,
    imaginary: T,
}


impl<T> Complex<T>
{
    fn new(real: T, imaginary: T) -> Complex<T>{
        Complex::<T> {real, imaginary}
    }
}

impl<T> Add for Complex<T>
    where T: Add<Output = T>
{
    type Output = Complex<T>;
    // adding 2 complex numbers returns a complex num
    fn add(self, rhs: Self) -> Self::Output
    {
        Complex{
            real: self.real + rhs.real,
            imaginary: self.imaginary + rhs.imaginary
        }
    }
}

let mut a = Complex::new(1.0, 2.0);
let mut b = Complex::new(3.0, 4.0);

println!("{:?}", a + b);

Complex { real: 4.0, imaginary: 6.0 }


* we can also implement the `AddAssign` operator for our Complex number

In [10]:
use std::ops::{AddAssign};

impl<T> AddAssign for Complex<T>
    where T: AddAssign<T>
{
    fn add_assign(&mut self, rhs: Self)
    {
        self.real += rhs.real;
        self.imaginary += rhs.imaginary;
    }
}

let mut a = Complex::new(1.0, 2.0);
let mut b = Complex::new(3.0, 4.0);

a += b;
println!("{:?}", a)

Complex { real: 4.0, imaginary: 6.0 }


()

* we can also implement the `Neg` operator for our Complex number

In [11]:
use std::ops::{Neg};

impl<T> Neg for Complex<T>
    where T: Neg<Output = T>
{
    type Output = Complex<T>;
    // negating a complex num
    fn neg(self) -> Self::Output
    {
        Complex{
            real: -self.real,
            imaginary: -self.imaginary
        }
    }
}

let mut a = Complex::new(1.0, 2.0);
println!("{:?}", -a)

Complex { real: -1.0, imaginary: -2.0 }


()

Comparibility

* with many custom data `struct`s, it is desirable to compare
  * comparibility allows them to be sorted in collection
  * these `trait`s can also be derived with `#[derive(PartialEq, Eq, PartialOrd, Ord)]`
* there are 2 types of equality in Rust
* 1. partial_eq and 2. (full) eq
  * full equality is not possible with float numbers
  * because of special cases like `NAN` = not a number (`0/0` or `inf/inf`)
  * also `NAN == NAN -> false`

In [12]:
// std::cmp::{par}
impl<T> PartialEq for Complex<T>
    where T: PartialEq
{
    fn eq(&self, rhs: &Self) -> bool
    {
        self.real == rhs.real && self.imaginary == rhs.imaginary
    }

    fn ne(&self, rhs: &Self) -> bool
    {
        self.real != rhs.real && self.imaginary != rhs.imaginary
    }
}

let mut a = Complex::new(1.0, 2.0);

println!("a==a is {}", a == a);

a==a is true


### 8.6 Static Dispatch

* dispatch is helping the compiler know what to call
* there are two kinds of dispatch in Rust:
  * 1 __static__: pre-determined
  * 2 __dynamic__: determined at run-time
* with static dispath, a function calls on a specific implementation of the variable
  * the type of variable has a implementation for this purpose
* in Rust, we can make use of __monomorphization__ [(source)](https://rustc-dev-guide.rust-lang.org/backend/monomorph.html)
  * Rust has a very expressive type system inluding generic types
  * but at run-time all variables need to have concrete type
  * to do so, Rust _monomorphizes_ all generic types
  * the compiler makes copies of the code for each specific type needed
  * this means faster programs but slower compile time
* for the example code, see also [trait objects](https://doc.rust-lang.org/reference/types/trait-object.html)

In [13]:
trait Printable
{ fn format(&self) -> String; }

impl Printable for i32
{
    fn format(&self) -> String{
        format!("i32: {}", *self)
    }
}

impl Printable for String
{
    fn format(&self) -> String{
        format!("string: {}", *self)
    }
}

// monomorphizable function
fn print_it<T: Printable>(z: T){
    println!("{}", z.format());
}

let a = 123;
let b = "hello".to_string();

print_it(a);
print_it(b)

i32: 123
string: hello


()

### 8.7 Dynamic Dispatch

* dynamic dispatch uses not a generic type but rather a reference
  * thus, the function call must be to referenced dynamic variables
    * `&` -> reference and `dyn` -> dynamic
  * the function will look for the specific implementation
  * it cannot be inferred at compile-time (because variables are not yet created)
  * the type is checked at run-time, which makes the function more expensive
    * opposite of the before (faster compile, slower run-time)

In [14]:
fn print_it_too(z: &dyn Printable){
    println!("{}", z.format());
}

let a = 123;
let b = "hello".to_string();

print_it_too(&a); // & -> reference to trait object
print_it_too(&b)

i32: 123
string: hello


()

### 8.8 Why Dynamic Dispatch

* from the discussion of pros and cons of static and dynamic dispatch, the safer and more efficient way seems to always use static dispatch
* but certain problems in OOP can only be solved witb dynamic dispatch
* example:
  * if various `struct`s (child classes) have the same `trait` (parent class)
  * when iterating over an array of these classes the compiler cannot know which specific `impl`ementation of the `trait` they posses
  * so the implemenation must be looked up dynamically for each specific `struct` at run-time

In [15]:
// reuse the shapes structs
// struct Circle {radius: f64,}
// struct Square {side: f64,}
#[derive(Debug)]
struct Triangle {a: f64, b: f64,}


// implement area for right triangle
impl Shape for Triangle {
    fn area(&self) -> f64 {
        self.a * self.b * 0.5}
}

In [16]:
fn demo_dyn_dispatch()
{   // array of 4 shapes
    let shapes:[&dyn Shape; 4] = [
        &Circle{radius: 1.0},
        &Square{side: 3.0},
        &Triangle{a: 2.0, b: 3.0},
        &Square{side: 4.0},
    ];

    for (i, shape) in shapes.iter().enumerate(){
        // during run-time the program look what is the shape
        // then it looks up the implementation for the area
        println!("Shape #{} has area {}", i, shape.area());
    }
}

demo_dyn_dispatch()

Shape #0 has area 3.141592653589793
Shape #2 has area 3
Shape #3 has area 16
Shape #1 has area 9


()

### 8.9 Vectors of Different Object

* what about vectors of different objects?
* there are two ways: 1. enums and 2. trait objects

_not working like it does in the lecture_

In [31]:
enum Creature{
    Human(Human),
    Cat(Cat),
}

let creatures = Vec::new();
creatures.push(Creature::Human(Human{name: "John"}));
creatures.push(Creature::Cat(Cat{name: "Fluffy"}));

for c in creatures{
    match c {
        Creature::Human(h) => h.talk(),
        Creature::Cat(k) => k.talk(),
    }
}

// let mut animals:Vec<Box<dyn Animal>> = Vec::new();
// animals.push(Box::new(Human{name: "John"}));
// animals.push(Box::new(Cat{name: "Fluffy"}));


Error: no variant or associated item named `new` found for enum `Creature` in the current scope